In [8]:
import numpy as np
import matplotlib.pyplot as plt
from BabelViscoFDTD.tools.RayleighAndBHTE  import ForwardSimple, GenerateSurface, SpeedofSoundWater
from BabelIntegrationBASE import (RUN_SIM_BASE, 
                            BabelFTD_Simulations_BASE,
                            SimulationConditionsBASE,
                            Material)

ModuleNotFoundError: No module named 'BabelIntegrationBASE'

In [ ]:


def GenerateFocusTx(f, Foc, Diam, c, PPWSurface=5):
    wavelength = c / f
    lstep = wavelength / PPWSurface
    Tx = GenerateSurface(lstep, Diam, Foc)
    return Tx

def CalculateRayleighFieldsFreeWater(self, deviceName='6800'):
    print("Calculating Rayleigh-based field in free water...")

    self._TxRC = self.GenTx()
    
    cwvnb_extlay = np.array(2 * np.pi * self._Frequency / SpeedofSoundWater(20.0) + 1j * 0).astype(np.complex64)
    
    u0 = (np.ones((self._TxRC['center'].shape[0], 1), np.float32) + 
          1j * np.zeros((self._TxRC['center'].shape[0], 1), np.float32)) * self._SourceAmpPa

    nxf = len(self._XDim)
    nyf = len(self._YDim)
    nzf = len(self._ZDim)
    yp, xp, zp = np.meshgrid(self._YDim, self._XDim, self._ZDim)
    rf = np.hstack((np.reshape(xp, (nxf * nyf * nzf, 1)),
                    np.reshape(yp, (nxf * nyf * nzf, 1)), 
                    np.reshape(zp, (nxf * nyf * nzf, 1)))).astype(np.float32)

    u2 = ForwardSimple(cwvnb_extlay, 
                       self._TxRC['center'].astype(np.float32),
                       self._TxRC['ds'].astype(np.float32), 
                       u0, rf, deviceMetal=deviceName)

    u2 = np.reshape(u2, xp.shape)
    self._u2RayleighField = u2  
    print("Simulation complete.")

frequency = 500e3  # Hz
aperture = 64e-3  # m
focal_length = 50e-3  # m
speed_of_sound = SpeedofSoundWater(20.0)  

class SimulationConditions(SimulationConditionsBASE):
    '''
    Simulation conditions for free water without CT or trajectory files.
    '''
    def __init__(self, Aperture=64e-3, FocalLength=50e-3, **kargs):
        super().__init__(Aperture=Aperture, FocalLength=FocalLength, **kargs)

    def GenTx(self):
        # Generate the transducer geometry
        c = SpeedofSoundWater(20.0)  # speed of sound in water
        return GenerateFocusTx(self._Frequency, self._FocalLength, self._Aperture, c)

    def CalculateRayleighFieldsFreeWater(self):
        print("Calculating Rayleigh-based field for CTX500 in free water...")

        # Generate transducer surface
        self._TxRC = self.GenTx()

        # Define the wave number
        cwvnb_extlay = np.array(2 * np.pi * self._Frequency / SpeedofSoundWater(20.0), dtype=np.complex64)

        # Initialize pressure field (complex amplitude)
        u0 = np.ones((self._TxRC['center'].shape[0], 1), dtype=np.complex64) * self._SourceAmpPa

        # Define the computational domain
        nx, ny, nz = len(self._XDim), len(self._YDim), len(self._ZDim)
        yp, xp, zp = np.meshgrid(self._YDim, self._XDim, self._ZDim)

        # Flatten domain to calculate the field
        rf = np.vstack((xp.ravel(), yp.ravel(), zp.ravel())).T.astype(np.float32)

        # Calculate the Rayleigh field
        u2 = ForwardSimple(cwvnb_extlay, self._TxRC['center'].astype(np.float32), 
                           self._TxRC['ds'].astype(np.float32), u0, rf)
        u2 = u2.reshape((nx, ny, nz))
        self._u2RayleighField = u2

        print("Field calculated in free water.")

sim_conditions = SimulationConditions(Frequency=frequency,
                                      Aperture=aperture,
                                      FocalLength=focal_length)

sim_conditions.CalculateRayleighFieldsFreeWater()

plt.imshow(np.abs(sim_conditions._u2RayleighField[:, :, int(len(sim_conditions._ZDim)/2)]),
           cmap='jet')
plt.colorbar()
plt.title('CTX500 in Free Water (Rayleigh Field)')
plt.show()


loading /Users/capricieuxv/anaconda3/envs/neuronav/lib/python3.8/site-packages/BabelViscoFDTD/tools/libBabelMetal.dylib
Metal devices
Apple M1 Pro
Is device low power? false.
Is device external? false.
Maximum threads per group: MTLSize(width: 1024, height: 1024, depth: 1024).
loaded Metal <CDLL '/Users/capricieuxv/anaconda3/envs/neuronav/lib/python3.8/site-packages/BabelViscoFDTD/tools/libBabelMetal.dylib', handle 7de22590 at 0x113dc5940>
Maximum buffer length: 8.0 GB.


ImportError: cannot import name 'SimulationConditions' from 'BabelViscoFDTD.tools.RayleighAndBHTE' (/Users/capricieuxv/anaconda3/envs/neuronav/lib/python3.8/site-packages/BabelViscoFDTD/tools/RayleighAndBHTE.py)